<a href="https://colab.research.google.com/github/PhantomXV7/PhantomXV7/blob/main/Amazon_ML_hackathon_TransformerModel_002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://drive.google.com/file/d/1--GuEHbUKqrztnRV9XLqXd7ru2aKwkfE/view?usp=share_link
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1--GuEHbUKqrztnRV9XLqXd7ru2aKwkfE' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1--GuEHbUKqrztnRV9XLqXd7ru2aKwkfE" -O train.ftr && rm -rf /tmp/cookies.txt
!pip install torch
!pip install transformers
!pip install -U sentence-transformers

--2023-04-22 19:38:12--  https://docs.google.com/uc?export=download&confirm=t&id=1--GuEHbUKqrztnRV9XLqXd7ru2aKwkfE
Resolving docs.google.com (docs.google.com)... 209.85.234.102, 209.85.234.113, 209.85.234.139, ...
Connecting to docs.google.com (docs.google.com)|209.85.234.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-28-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ojaebdsfbfhp8ps4npc3h0tv7qb8vf7l/1682192250000/12978060410603387978/*/1--GuEHbUKqrztnRV9XLqXd7ru2aKwkfE?e=download&uuid=e56bc7ea-2952-48a6-b936-82cb4e0cd0d4 [following]
--2023-04-22 19:38:12--  https://doc-0s-28-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ojaebdsfbfhp8ps4npc3h0tv7qb8vf7l/1682192250000/12978060410603387978/*/1--GuEHbUKqrztnRV9XLqXd7ru2aKwkfE?e=download&uuid=e56bc7ea-2952-48a6-b936-82cb4e0cd0d4
Resolving doc-0s-28-docs.googleusercontent.com (doc-0s-28-docs.googleusercontent.com)... 172.217.212.132,

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
from tqdm import tqdm
import sys
import locale
import shutil

# Set locale to UTF-8
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

In [44]:
import locale
print(locale.getpreferredencoding())
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

UTF-8


In [45]:
from tqdm.notebook import tqdm
tqdm.pandas()


In [46]:
gc.collect()

2805

In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [59]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
# sentences = ['This is an example sentence', 'Each sentence is converted']

# # Load model from HuggingFace Hub
# tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
# model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
# # print(model)
# # Tokenize sentences
# encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# print(encoded_input)

# # # Compute token embeddings
# # with torch.no_grad():
# #     model_output = model(**encoded_input)

# # # Perform pooling
# # sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# # # Normalize embeddings
# # sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

# # print("Sentence embeddings:")
# # print(sentence_embeddings)


{'input_ids': tensor([[ 101, 2023, 2003, 2019, 2742, 6251,  102],
        [ 101, 2169, 6251, 2003, 4991,  102,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0]])}


In [49]:
class SentenceTextRegressor(nn.Module):
    def __init__(self, model, model_out):
        super(SentenceTextRegressor, self).__init__()
        self.model = model 
        self.fc = nn.Linear(in_features=model_out, out_features=1)
        self.dropout = nn.Dropout(p = 0.33)
        self.relu = nn.ReLU()
        
    def forward(self, X):
        model_features = model(**X)
        sentence_embeddings = mean_pooling(model_features, X['attention_mask'])
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        regressits = self.relu(self.dropout(self.fc(sentence_embeddings)))
        return regressits

In [60]:
class CustomDataset(Dataset):
    def __init__(self, df, tokenizer = None):
        self.df = df
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.df['td'][index]
        encoded_text = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        #encoded_text = torch.tensor(encoded_text)
        if 'PRODUCT_LENGTH' in self.df.columns:
            y = torch.tensor(self.df['PRODUCT_LENGTH'][index])
            
            return encoded_text, y
        else:
            return encoded_text

In [51]:
config = {
    'train_path' : 'train.ftr',
    'test_path' : 'test.ftr',
    'tokenizer' : 'AutoTokenizer',
    'input_dim' : 10,
    'fusion_out' : 128,
    'embed_dim' : 128,
    'token_len' : 128,
    'model' : 'Sadge'
}

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [52]:
model = SentenceTextRegressor(model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2'), model_out  = 384)
print(model)
model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-6)

SentenceTextRegressor(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-12, elementw

In [53]:
train_df = pd.read_feather('train.ftr') 

In [54]:
from tqdm.notebook import tqdm

tqdm.pandas()

train_df = train_df.sort_values(by = ['PRODUCT_LENGTH'], ascending = True).iloc[:, :len(train_df)//10*7]
train_df['td'] = train_df['td'].progress_apply(lambda x : x if len(x.split()) <= 64 else ' '.join(x.split()[:64]))

remove_n = int(0.3*len(train_df))
drop_indices = np.random.choice(train_df.index, remove_n, replace=False)
train_df.drop(drop_indices, inplace = True)


  0%|          | 0/2249698 [00:00<?, ?it/s]

In [55]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(train_df, train_size = 0.8, shuffle=True)
train_df.reset_index(inplace = True)
val_df.reset_index(inplace = True)

In [56]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
train_dataset = CustomDataset(df = train_df, tokenizer = tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [57]:

best_loss = float('inf')  # Initialize with a large value
best_model = None
num_epochs = 1
for epoch in range(num_epochs):
    batch_num = 1
    for batch, y in tqdm(train_dataloader):
        inputs = batch.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        if loss.item() < best_loss:
            best_loss = loss.item()
            best_model = model.state_dict()
        
        batch_num+=1
        if batch_num%100 == 0:
           print(f'Loss: {loss.item()}')
        
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')



/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2372: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


RuntimeError: ignored

In [ ]:
torch.save(best_model, 'best_model.pth')

In [ ]:
del train
del train_dataset
del train_dataloader
gc.collect()

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Lqia2upHkEE9Bu67eMJqTskelN8JV4__' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Lqia2upHkEE9Bu67eMJqTskelN8JV4__" -O test.ftr && rm -rf /tmp/cookies.txt

In [ ]:
test_df = pd.read_feather('test.ftr')
test_df['td'] = test_df['td'].progress_apply(lambda x : x if len(x.split()) <= 64 else ' '.join(x.split()[:64]))

test_dataset = CustomDataset(test_df, tokenizer = tokenizer)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
results = []
model.eval()
with torch.no_grad():
    for batch in test_dataloader:
        inputs = batch.to(device)
        outputs = model(inputs)
        results.append(outputs.to('cpu'))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import locale
# import shutil

# # Set locale to UTF-8
# locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# # Copy file to destination
# shutil.copy2('/content/train_cleaned.ftr', '/content/drive/MyDrive/')


In [ ]:
# !cp -r '/content/train_cleaned.ftr' /content/drive/MyDrive/